In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pandas as pd

df = pd.read_csv("../products.csv")
print(df.shape)
print(df.columns.tolist())

(88970, 44)
['product_id', 'product_name', 'product_url', 'size', 'net_weight', 'purity', 'supplier_wastage', 'regular_wastage', 'premium_wastage', 'description', 'metal_type', 'availability', 'style', 'sub_style', 'delivery_timeline', 'gross_weight', 'cz_stone_weight', 'solitaire_stone_weight', 'color_stone_weight', 'moissanite_weight', 'moti_weight', 'kundan_weight', 'rudraksh_weight', 'diamond_weight', 'labour_charge', 'brand_or_factory_name', 'supplier_name', 'tunch', 'no_of_pieces', 'no_of_stone_pieces', 'diamond_quality', 'diamond_size', 'color', 'vendor_id', 'vendor_sku', 'global_sku', 'stone_approach', 'curated_status_flag', 'active_status_flag', 'featured_product', 'variant_price_base', 'variant_price_sub', 'category_name', 'sub_category_name']


/var/folders/pq/bmw3qp553sq2x9k4vmz_v7rr0000gn/T/ipykernel_47638/4278171981.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../products.csv")


In [7]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///products.db")
df.to_sql("products", engine, index=False)

88970

In [8]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['products']


In [32]:
import ast
import re

In [33]:
def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

In [39]:
def query_as_list(db, query):
    res = db.run(query)  # Run the query
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", str(string)).strip() for string in res]
    return list(set(res)) 

In [41]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = query_as_list(db, query)
table_info = {}
for table in tables:
    query = f"PRAGMA table_info({table});"
    columns = query_as_list(db, query)
    table_info[table] = columns
table_info

{'products': ['',
  'FLOAT',
  'style',
  'no_of_stone_pieces',
  'purity',
  'size',
  'TEXT',
  'sub_category_name',
  'premium_wastage',
  'description',
  'curated_status_flag',
  'diamond_size',
  'sub_style',
  'featured_product',
  'product_id',
  'moti_weight',
  'diamond_quality',
  'variant_price_sub',
  'kundan_weight',
  'vendor_id',
  'supplier_wastage',
  'color',
  'category_name',
  'delivery_timeline',
  'global_sku',
  'BIGINT',
  'stone_approach',
  'net_weight',
  'variant_price_base',
  'diamond_weight',
  'brand_or_factory_name',
  'no_of_pieces',
  'product_url',
  'color_stone_weight',
  'tunch',
  'cz_stone_weight',
  'labour_charge',
  'active_status_flag',
  'gross_weight',
  'solitaire_stone_weight',
  'moissanite_weight',
  'rudraksh_weight',
  'supplier_name',
  'metal_type',
  'vendor_sku',
  'regular_wastage',
  'product_name',
  'availability']}

In [31]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = db.run(query)
# table_info = {}
# for table in tables:
#     query = f"PRAGMA table_info({table});"
#     columns = db.run(query)
#     table_info[table] = columns
#     table_info
type(tables)

str

In [26]:
o = db.run("SELECT name FROM sqlite_master WHERE type='table';")
print(o)


[('products',)]


In [20]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [21]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [23]:
agent_executor.invoke({"input": "tell me some different types of bangles"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


products
Invoking: `sql_db_schema` with `{'table_names': 'products'}`



CREATE TABLE products (
	product_id TEXT, 
	product_name TEXT, 
	product_url TEXT, 
	size TEXT, 
	net_weight FLOAT, 
	purity TEXT, 
	supplier_wastage FLOAT, 
	regular_wastage FLOAT, 
	premium_wastage FLOAT, 
	description FLOAT, 
	metal_type BIGINT, 
	availability TEXT, 
	style FLOAT, 
	sub_style FLOAT, 
	delivery_timeline TEXT, 
	gross_weight FLOAT, 
	cz_stone_weight FLOAT, 
	solitaire_stone_weight FLOAT, 
	color_stone_weight FLOAT, 
	moissanite_weight FLOAT, 
	moti_weight FLOAT, 
	kundan_weight FLOAT, 
	rudraksh_weight FLOAT, 
	diamond_weight FLOAT, 
	labour_charge FLOAT, 
	brand_or_factory_name FLOAT, 
	supplier_name TEXT, 
	tunch FLOAT, 
	no_of_pieces TEXT, 
	no_of_stone_pieces FLOAT, 
	diamond_quality FLOAT, 
	diamond_size FLOAT, 
	color TEXT, 
	vendor_id FLOAT, 
	vendor_sku TEXT, 
	global_sku TEXT, 
	stone_approach FLOAT,

{'input': 'tell me some different types of bangles',
 'output': 'Here are some different types of bangles available:\n\n1. Turkey Bangles\n2. TAJ-1462-D\n3. OLVSBGSC-0043\n4. AVY125-1094-C\n5. BL-291\n6. SE-1796-A\n7. AVY125-1663\n\nIf you need more information or more examples, feel free to ask!'}

# Agent

In [24]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x159254d60>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x159254d60>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x159254d60>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [25]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
prompt_template.messages[0].pretty_print()

/Users/a2024/miniforge3/envs/generativeai/lib/python3.13/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [26]:
system_message = prompt_template.format(dialect="SQLite", top_k=15)

In [27]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [28]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


product = query_as_list(db, "SELECT product_name FROM products")
category_name = query_as_list(db, "SELECT category_name FROM products")
category_name[:5]

['Stud', 'Leaf Chain', 'Premium Pipe Bali', 'Casting Bali', 'Chapela Pendant']

In [29]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [30]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [31]:
from langchain.agents.agent_toolkits import create_retriever_tool

_ = vector_store.add_texts(product + category_name)
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [32]:
# Add to system message
suffix = (
    "If you need to filter on a proper noun like a Name, you must ALWAYS first look up "
    "the filter value using the 'search_proper_nouns' tool! Do not try to "
    "guess at the proper name - use this function to find similar ones."
    

)
prefix = (
        "You are a customer service CHATBOT your name is ANTHER AI"
        "using the sql tools generate the relevent answer so be consise and clear in your responses. Be polite and helpful."
)

system = f"{prefix}\n\n{system_message}\n\n{suffix}"

tools.append(retriever_tool)


# agent = create_react_agent(llm, tools, state_modifier=system)

In [135]:
system

"You are a customer service CHATBOT your name is ANTHER AIusing the sql tools generate the relevent answer so be consise and clear in your responses. Be polite and helpful.\n\nSystem: You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 15 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewri

In [33]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [38]:
question = "give the no of chain types and their names"    

for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values", config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

give the no of chain types and their names
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_ruwfqRTOkuyCbX7u6X8JPDrV)
 Call ID: call_ruwfqRTOkuyCbX7u6X8JPDrV
  Args:
    query: SELECT DISTINCT category_name FROM products WHERE category_name LIKE '%chain%';
================================= Tool Message =================================
Name: sql_db_query

[('Chains',), ('Handmade Chain',), ('Chains Style Cocktail',), ('Casting Chain',), ('Casting Chain Sets',), ('Uthappa Chain',), ('Casting Chain Pendant Sets',), ('Ladies Chain',), ('Chain Sets',), ('Casting Ladies Chain',), ('Nawabi Chain',), ('Ladies Chains',), ('Gents Chain',), ('Leaf Chain',), ('Fancy Pipe Chain',), ('Indo Italy Chain',), ('Nice Machine Chains',), ('Casting Chain Pendant',), ('Bahubali Chains',), ('Super Hollow Chains',), ('Hollow Nawabi Chains',), ('Chain Pendant',), ('